In [25]:
import json
import pandas as pd
from plotly import*
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto

app = dash.Dash(__name__)
server = app.server

table = "network_actualized_for_forecast (1) (2).json"

with open(table, 'r', encoding='utf-8') as f:
    table = json.load(f)

class Model():
    
    def add_element():
        id_source = int(input('Введите id пласта от 126 до 238\n'))

        if (126 <= id_source <= 238):
            for  pipes1 in table["pipes"]:
                if  id_source == pipes1["id"] and pipes1["type"] == "WELL": 
                    #print(f"SOURCE ID: {pipes['id']} --> WELL ID: {pipes['id']}")  
                    a = pipes1['id']

            for  pipes1  in table["pipes"]:
                for  pipes2 in table["pipes"]:
                    if (pipes1["id"] == id_source and pipes1["type"] == "WELL") and (pipes2["type"] == "CONNECTOR") and pipes1["outletNodeId"] == pipes2["inletNodeId"]:
                        #print(f"SOURCE ID: {pipes['id']} --> WELL ID: {pipes['id']} --> CONNECTOR ID: {pipes1['id']}")  
                        b = pipes2['id']

            for  pipes2 in table["pipes"]:
                for  pipes3  in table["pipes"]:
                    if (pipes2["id"] == b and pipes2["type"] == "CONNECTOR") and (pipes3["type"] == "TUBE") and pipes2["outletNodeId"] == pipes3["inletNodeId"]:
                        #print(f"SOURCE ID: {pipes['id']} --> WELL ID: {pipes['id']} --> CONNECTOR ID: {pipes1['id']}")  
                        c = pipes3['id']
                        #print(f"SOURCE ID: {a} --> WELL ID: {a} --> CONNECTOR ID: {b} --> JUNCTION ID: {b} --> TUBE ID: {c} --> SINK ID: 113")
            
            
            data = ({'from':[f"SOURCE ID: {a}",f"JUNCTION ID: {b}"],
                      'to':[f"JUNCTION ID: {b}",f"SINK ID: 113"]})
            edges = pd.DataFrame(data)
            nodes = set()

            cy_edges = []
            cy_nodes = []

            for index, row in edges.iterrows():
                source, target = row['from'], row['to']

                if source not in nodes:
                    nodes.add(source)
                    cy_nodes.append({"data": {"id": source, "label": source}})
                if target not in nodes:
                    nodes.add(target)
                    cy_nodes.append({"data": {"id": target, "label": target}})

                cy_edges.append({
                    'data': {
                        'source': source,
                        'target': target
                    }
                })

            stylesheet = [
                {
                    "selector": 'node', #For all nodes
                    'style': {
                        "opacity": 0.9,
                        "label": "data(label)", #Label of node to display
                        "background-color": "#07ABA0", #node colorg

                    }
                },
                {
                    "selector": 'edge', #For all edges
                    "style": {

                        "label": f"TUBE {c}",
                        "textposition": "top",
                        "target-arrow-color": "#8A0C40", #Arrow color
                        "target-arrow-shape": "triangle", #Arrow shap
                        "line-color": "#C5D3E2", #edge color
                        'arrow-scale': 2, #Arrow size
                        'curve-style': 'bezier' #Default curve-If it is style, the arrow will not be displayed, so specify it
                },
                
            }]
            

            app.layout = html.Div([
                dcc.Dropdown(
                        id='dropdown-layout',
                        options=[
                            {'label': 'random',
                             'value': 'random'},
                            {'label': 'grid',
                             'value': 'grid'},
                            {'label': 'circle',
                             'value': 'circle'},
                            {'label': 'concentric',
                             'value': 'concentric'},
                            {'label': 'breadthfirst',
                             'value': 'breadthfirst'},
                            {'label': 'cose',
                             'value': 'cose'}
                        ], value='grid'
                    ),
                html.Div(children=[
                    cyto.Cytoscape(
                        id='cytoscape',
                        elements=cy_edges + cy_nodes,
                        style={
                            'height': '95vh',
                            'width': '100%'
                        },
                        stylesheet=stylesheet #Give the style you just defined
                    )
                ])

            ])

            @app.callback(Output('cytoscape', 'layout'),
                        [Input('dropdown-layout', 'value')])
            def update_cytoscape_layout(layout):
                return {'name': layout}

            if __name__ == '__main__':
                app.run_server(debug=False)

        else:
            print('ERROR')

    def get_element():
        id = int(input('Введите id'))
        type = str(input('Введите type(Caps Lock)'))

        for Node in table["nodes"]:
            if (0<=id<=113):
                if id == Node["id"] and type == Node["type"] and Node["type"] == "JUNCTION":
                    print(f"id:{Node['id']}\n type:{Node['type']}\n name:{Node['name']}\n on:{Node['on']}\n gasRateTonneToD:{Node['gasRateTonneToD']}\n  oilRateTonneToD:{Node['oilRateTonneToD']}\n waterRateTonneToD:{Node['waterRateTonneToD']}\n")
                elif id == Node["id"] and Node["type"] == "SINK":
                    print(f"id:{Node['id']}\n type:{Node['type']}\n name:{Node['name']}\n inletPressureAtma:{Node['inletPressureAtma']}\n")
               

            if (114<=id<=125):
                if id == Node["id"] and type == Node["type"] and Node["type"] == "JUNCTION":
                    print(f"id:{Node['id']}\n type:{Node['type']}\n name:{Node['name']}\n")
            

            if (126<=id<=238):
                if id == Node["id"] and type == Node["type"] and Node["type"] == "SOURCE":
                    print(f"id:{Node['id']}\n type:{Node['type']}\n name:{Node['name']}\n on:{Node['on']}\n gasRateTonneToD:{Node['gasRateTonneToD']}\n  oilRateTonneToD:{Node['oilRateTonneToD']}\n waterRateTonneToD:{Node['waterRateTonneToD']}\n")
             
                
       
        for Pipes in table["pipes"]:
            if (0<=id<=6):
                if id == Pipes["id"] and type == Pipes["type"] and Pipes["type"] == "TUBE":
                    print(f"id:{Pipes['id']}\n type:{Pipes['type']}\n name:{Pipes['name']}\n inletNodeId:{Pipes['inletNodeId']}\n outletNodeId:{Pipes['outletNodeId']}\n innerDiameterMm:{Pipes['innerDiameterMm']}\n profileHorDistanceMSpaceHeightM:{Pipes['profileHorDistanceMSpaceHeightM']}\n roughnessMm:{Pipes['roughnessMm']}\n")
            
            if (7<=id<=11):
                if id == Pipes["id"] and type == Pipes["type"] and Pipes["type"] == "TUBE":
                    print(f"id:{Pipes['id']}\n type:{Pipes['type']}\n name:{Pipes['name']}\n inletNodeId:{Pipes['inletNodeId']}\n outletNodeId:{Pipes['outletNodeId']}\n")
             

            if (12<=id<=124):
                if id == Pipes["id"] and type == Pipes["type"] and Pipes["type"] == "CONNECTOR":
                    print(f"id:{Pipes['id']}\n type:{Pipes['type']}\n name:{Pipes['name']}\n inletNodeId:{Pipes['inletNodeId']}\n outletNodeId:{Pipes['outletNodeId']}\n")
                elif id == Pipes["id"] and type == Pipes["type"] and Pipes["type"] == "TUBE":
                    print(f"id:{Pipes['id']}\n type:{Pipes['type']}\n name:{Pipes['name']}\n inletNodeId:{Pipes['inletNodeId']}\n outletNodeId:{Pipes['outletNodeId']}\n innerDiameterMm:{Pipes['innerDiameterMm']}\n profileHorDistanceMSpaceHeightM:{Pipes['profileHorDistanceMSpaceHeightM']}\n roughnessMm:{Pipes['roughnessMm']}\n")
             

            if (125<=id<=237):
                if id == Pipes["id"] and type == Pipes["type"] and Pipes["type"] == "WELL":
                    print(f"id:{Pipes['id']}\n type:{Pipes['type']}\n name:{Pipes['name']}\n inletNodeId:{Pipes['inletNodeId']}\n outletNodeId:{Pipes['outletNodeId']}\n VFPModelNumber:{Pipes['VFPModelNumber']}\n")
                

print(Model.add_element())

print(Model.get_element())
print(Model.get_element())
print(Model.get_element())
print(Model.get_element())
print(Model.get_element())
print(Model.get_element())


None
